# Steel industry energy consumption forecasting

_by Virginia Herrero_

## Introduction

The steel industry is crucial to modern manufacturing but is also a major consumer of energy, leading to high operational costs and environmental impacts. As demand for steel rises, optimizing energy consumption becomes increasingly urgent.

This machine learning project focuses on analyzing energy consumption patterns within the steel industry, focusing on data from DAEWOO Steel Co. Ltd in Gwangyang, South Korea, which produces various coils, steel plates, and iron plates. By leveraging historical data and operational parameters, the project aims to identify key factors influencing energy use and develop predictive models to enhance energy efficiency. Ultimately, this initiative seeks to provide actionable insights that promote sustainability and reduce the carbon footprint of steel production.

## Project Overview

* Data loading
* Data cleaning
* Data exploration
* Pre-model data transformation
* Linear regression model

## Data loading
Load the CSV file **steel-industry-data** as a pandas DataFrame.

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Load the dataset
df = pd.read_csv("steel-industry-data.csv")
df.head()

## Data cleaning
Clean and pre-process the dataset prior to conducting further analysis.

In [ ]:
df.info()

* **Remove unnecessary columns**

The date and NSM columns are not necessary for this analysis, as it is not a time series analysis. Therefore, they are removed from the dataset.

In [ ]:
df = df.drop("date", axis = 1)

* **Rename columns**

Some column names have been renamed to enhance readability and improve comprehension of the dataset.

In [ ]:
df.columns

In [ ]:
df = df.rename(columns = {"Usage_kWh" : "energy_usage_kWh",
                          "Lagging_Current_Reactive.Power_kVarh" : "lagging_current_kVarh",
                          "Leading_Current_Reactive_Power_kVarh" : "leading_current_kVarh",
                          "CO2(tCO2)" : "CO2_ppm",
                          "Lagging_Current_Power_Factor" : "lagging_current_power_factor",
                          "Leading_Current_Power_Factor" : "leading_current_power_factor",
                          "WeekStatus" : "week_status",
                          "Day_of_week" : "day_of_the_week",
                          "Load_Type" : "load_type"})

In [ ]:
df.head()

* **Data types**

Check that all columns have the appropriate data types.

In [ ]:
df.dtypes

* **Null values**

Identify and remove any missing values, zero values, or NaN values from the dataset as needed.

In [ ]:
# Check the total of null values in each column
df.isna().sum()

There are no missing values in the dataset.

* **Duplicated values**

Verify if there are any duplicate entries in the dataset.

In [ ]:
df.duplicated().sum()

In [ ]:
# Show all duplicated values
df[df.duplicated()]

In [ ]:
# Drop all duplicated values
df = df.drop_duplicates()

Eighty-one duplicate entries were detected and removed from the dataset.

* **Outliers**

Check for outliers in the dataset by first examining its statistical summary. This will provide an initial overview of the data.

In [ ]:
df.describe()

At first glance, it can be inferred that there are some outliers in the columns **energy_usage_kWh**, **lagging_current_kVarh**, and **leading_current_kVarh**. This conclusion is based on the observation that the maximum values exceed both the mean and the median, which can indicate the presence of outliers.

A more thorough evaluation of these outliers is necessary to identify them and assess the appropriate approach for handling them.

Outliers in these three columns or features will be identified using the Interquartile Range (IQR) method. Analyzing the distribution of the data will reveal values that fall outside the typical range, enabling the selection of an appropriate approach for handling these outliers.

**1. Outliers in the feature "energy_usage_kWh":**

In [ ]:
# Plot the energy usage distribution using a histogram and boxplot
energy_usage_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["energy_usage_kWh"], ax = ax[0], bins = 15, binrange = (0, 150), color = "#41b6c4")
sns.boxplot(x = df["energy_usage_kWh"], ax = ax[1], color = "#41b6c4")
ax[0].set_xlabel("Energy Usage (kWh)")
ax[1].set_xlabel("Energy Usage (kWh)")
plt.suptitle("Energy Usage Distribution", size = 12)

A significant number of outliers have been identified through the boxplot of the feature "energy_usage_kWh." These outliers will be now quantified statistically.

In [ ]:
# Create a function to find outliers using the IQR method

def find_outliers_iqr(dataframe, column):
    """
    Finds outliers in the specified column of a DataFrame using the IQR method

    Parameters
    ----------
    dataframe : Pandas DataFrame
        The DataFrame containing the data
    
    column : str
        The name of the column (as a string) in which to find the outliers

    Returns
    -------
    Pandas DataFrame
        A DataFrame containing the outliers identified in the specified column
   """
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    
    # Calculate IQR
    IQR = Q3 - Q1
    
    # Determine the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers
    outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
    print(f"The number of outliers in the column {column} is {outliers.shape[0]}")
    
    return outliers

In [ ]:
outliers_energy_usage = find_outliers_iqr(df, "energy_usage_kWh")
outliers_energy_usage

The number of outliers is relatively low, so while they could simply be removed, I’ve decided to cap them instead. This approach allows me to retain the overall structure of the dataset while minimizing the impact of extreme values. 

In [ ]:
def cap_outliers(dataframe, column):
    """
    Cap outliers in a specified column of the DataFrame using the IQR method

    Parameters
    ----------
    dataframe : Pandas DataFrame
        The DataFrame containing the data
    
    column : str
        The name of the column to cap outliers


    Returns
    -------
    Pandas DataFrame
        A DataFrame with outliers capped
    """
    if column not in dataframe.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame.")
    
    # Calculate Q1 and Q3
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)

    # Calculate IQR

    IQR = Q3 - Q1

    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Cap the outliers
    dataframe[column] = dataframe[column].clip(lower = lower_bound, upper = upper_bound)

    return dataframe

In [ ]:
df = cap_outliers(df, "energy_usage_kWh")

**2. Outliers in the feature "lagging_current_kVarh":**

In [ ]:
# Plot the lagging current distribution using a histogram and boxplot
lagging_current_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["lagging_current_kVarh"], ax = ax[0], bins = 8, binrange = (0, 80), color = "#41b6c4")
sns.boxplot(x = df["lagging_current_kVarh"], ax = ax[1], color = "#41b6c4")
ax[0].set_xlabel("Lagging current (kVarh)")
ax[1].set_xlabel("Lagging current (kVarh)")
plt.suptitle("Lagging Current Distribution", size = 12)

In [ ]:
outliers_lagging_current = find_outliers_iqr(df, "lagging_current_kVarh")
outliers_lagging_current

The feature "lagging_current_kVarh" has more outliers than the energy usage feature. These outliers will be addressed in a consistent manner, with capping applied at the upper limit determined by the IQR method.

In [ ]:
df = cap_outliers(df, "lagging_current_kVarh")

**3. Outliers in the feature "leading_current_kVarh":**

In [ ]:
# Plot the lagging current distribution using a histogram and boxplot
leading_current_distribution = plt.figure()
fig, ax = plt.subplots(1, 2, figsize = (11, 3))
sns.histplot(df["leading_current_kVarh"], ax = ax[0], bins = 6, binrange = (0, 30), color = "#41b6c4")
sns.boxplot(x = df["leading_current_kVarh"], ax = ax[1], color = "#41b6c4")
ax[0].set_xlabel("Leading current (kVarh)")
ax[1].set_xlabel("Leading current (kVarh)")
plt.suptitle("Leading Current Distribution", size = 12)

In [ ]:
outliers_leading_current = find_outliers_iqr(df, "leading_current_kVarh")
outliers_leading_current

The number of outliers in the feature "leading_current_kVahr" is significantly greater than in the other features. Nevertheless, the approach to handling these outliers will remain consistent.

In [ ]:
df = cap_outliers(df, "leading_current_kVarh")

**The clean dataset:**

In [ ]:
energy_consumption = df.copy()
energy_consumption.head()

## Data exploration

In this section, an in-depth data exploratory analysis is carried out.

* **Univariate exploration**

Let's have a quick glance to the whole dataset feature by feature to have a general overview of the dataset.

In [ ]:
# Plot all features distribution
energy_consumption.hist(figsize = (20, 8), color = "#41b6c4")
plt.suptitle("Feature Distribution")
plt.show()

* **Correlation matrix**

A correlation matrix is used to find the correlation between features.

In [ ]:
correlations = energy_consumption.corr(numeric_only = True)
correlations

In [ ]:
correlation_heatmap_graph = plt.figure(figsize = (7, 3))
sns.heatmap(correlations, linewidths = 0.5, annot = True, cmap = "YlGnBu")
plt.title("Correlation Heatmap", size = 12)

Here’s a brief explanation of the insights obtained from the correlation matrix:

### Strong Positive Correlations:
* **Energy Usage and Lagging Current**:
   - As energy usage increases, lagging current also tends to increase. This suggests that higher energy consumption is associated with more inductive loads, which typically exhibit lagging current.

* **Energy Usage and CO2**:
   - Higher energy usage correlates with increased CO2 emissions. This relationship likely reflects the reliance on fossil fuels for energy, which emit CO2 when consumed.

* **Lagging Current and CO2**:
   - A strong correlation indicates that as lagging current increases, CO2 emissions also increase. This may suggest that systems with higher inductive loads (and thus higher lagging currents) are contributing more to CO2 emissions.

### Moderate Positive Correlation:
* **Energy Usage and Lagging and Leading Current Power**:
   - This indicates that as energy usage increases, both lagging and leading current power also increase. It suggests that both types of power contribute to overall energy consumption, reflecting different load characteristics.

### Strong Negative Correlation:
* **Leading Current and Leading Current Power Factor**:
   - A strong negative correlation means that as leading current increases, the leading current power factor decreases. This implies that higher leading current is associated with less efficient power usage, potentially due to reactive power becoming more prominent.

### Moderate Negative Correlations:
* **Energy Usage and Leading Current**:
   - As energy usage increases, leading current tends to decrease. This might indicate that systems with higher energy consumption rely less on capacitive loads.

* **Lagging Current and Leading Current**:
   - This negative correlation suggests that as lagging current increases, leading current decreases, reflecting the competing nature of inductive and capacitive loads in a system.

* **Leading Current and CO2**:
   - Higher leading current is associated with lower CO2 emissions. This might indicate that systems using more capacitive loads (which can produce leading currents) are emitting less CO2, possibly due to greater efficiency or reliance on cleaner energy sources.

After identifying the correlations between the features of the dataset, the next step is to determine the type of correlation: linear or non-linear. To achieve this, the findings from the correlation heatmap will be visualized using scatter plots.

In [ ]:
# Plot scatterplots
correlation_scatterplot_graph = plt.figure()

fig, ax = plt.subplots(5, 2, figsize=(12, 30))

ax = ax.flatten()

sns.scatterplot(data = energy_consumption, x = "energy_usage_kWh", y = "lagging_current_kVarh", ax = ax[0], color = "#41b6c4")
ax[0].set_title("Energy Usage vs Lagging Current")
ax[0].set_xlabel("Energy Usage (kWh)")
ax[0].set_ylabel("Lagging Current (kVarh)")

sns.scatterplot(data = energy_consumption, x = "energy_usage_kWh", y = "leading_current_kVarh", ax = ax[1], color = "#0c2c84")
ax[1].set_title("Energy Usage vs Leading Current")
ax[1].set_xlabel("Energy Usage (kWh)")
ax[1].set_ylabel("Leading Current (kVarh)")

sns.scatterplot(data = energy_consumption, x = "energy_usage_kWh", y = "CO2_ppm", ax = ax[2], color = "#41b6c4")
ax[2].set_title("Energy Usage vs CO$_2$ emissions")
ax[2].set_xlabel("Energy Usage (kWh)")
ax[2].set_ylabel("CO$_2$ (ppm)")

sns.scatterplot(data = energy_consumption, x = "lagging_current_kVarh", y = "CO2_ppm", ax = ax[3], color = "#0c2c84") 
ax[3].set_title("Lagging Current vs CO$_2$ emissions")
ax[3].set_xlabel("Lagging Current (kVarh)")
ax[3].set_ylabel("CO$_2$ (ppm)")

sns.scatterplot(data = energy_consumption, x = "leading_current_kVarh", y = "CO2_ppm", ax = ax[4], color = "#41b6c4") 
ax[4].set_title("Leading Current vs CO$_2$ emissions")
ax[4].set_xlabel("Leading Current (kVarh)")
ax[4].set_ylabel("CO$_2$ (ppm)")

sns.scatterplot(data = energy_consumption, x = "lagging_current_kVarh", y = "leading_current_kVarh", ax = ax[5], color = "#0c2c84") 
ax[5].set_title("Lagging Current vs Leading Current")
ax[5].set_xlabel("Lagging Current (kVarh)")
ax[5].set_ylabel("Leading Current (kVarh)")

sns.scatterplot(data = energy_consumption, x = "energy_usage_kWh", y = "lagging_current_power_factor", ax = ax[6], color = "#41b6c4") 
ax[6].set_title("Energy Usage vs Lagging Current Power Factor")
ax[6].set_xlabel("Energy Usage (kWh)")
ax[6].set_ylabel("Lagging Current Power Factor")

sns.scatterplot(data = energy_consumption, x = "energy_usage_kWh", y = "leading_current_power_factor", ax = ax[7], color = "#0c2c84") 
ax[7].set_title("Energy Usage vs Leading Current Power Factor")
ax[7].set_xlabel("Energy Usage (kWh)")
ax[7].set_ylabel("Leading Current Power Factor")

sns.scatterplot(data = energy_consumption, x = "leading_current_kVarh", y = "leading_current_power_factor", ax = ax[8], color = "#41b6c4") 
ax[8].set_title("Leading Current vs Leading Current Power Factor")
ax[8].set_xlabel("Leading Current (kVarh)")
ax[8].set_ylabel("Leading Current Power Factor")

sns.scatterplot(data = energy_consumption, x = "lagging_current_kVarh", y = "leading_current_power_factor", ax = ax[9], color = "#0c2c84") 
ax[9].set_title("Lagging Current vs Leading Current Power Factor")
ax[9].set_xlabel("Lagging Current (kVarh)")
ax[9].set_ylabel("Leading Current Power Factor")

plt.show()

The scatter plots indicate that all the correlations are linear.

* **Energy consumption during the week**

The days with the highest energy consumption are Tuesdays and Thursdays, while Sundays show the lowest consumption.

In [ ]:
consumption_during_the_week_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "day_of_the_week", y = "energy_usage_kWh", errorbar= None, hue = "day_of_the_week", palette = "YlGnBu")
plt.xlabel("Day of the Week")
plt.ylabel("Energy Usage (kWh)")
plt.title("Energy Consumption During the Week", size = 12)

Energy consumption varies between weekdays and weekends, with higher usage typically observed on weekdays and lower consumption on weekends.

In [ ]:
consumption_by_weekstatus_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "week_status", y = "energy_usage_kWh", errorbar= None, hue = "week_status", palette = "YlGnBu")
plt.xlabel("Week Status")
plt.ylabel("Energy Usage (kWh)")
plt.title("Energy Consumption During the Week", size = 12)

* **Energy consumption by load type**

Energy consumption is higher during maximum load conditions and lower when the load type is light.

In [ ]:
consumption_by_load_type_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "load_type", y = "energy_usage_kWh", errorbar= None, hue = "load_type", palette = "YlGnBu")
plt.xlabel("Load Type")
plt.ylabel("Energy Usage (kWh)")
plt.title("Energy Consumption by Load Type", size = 12)

* **Energy consumption by load during the week**

The distribution of energy consumption based on load type remains consistent throughout the week.

In [ ]:
consumption_by_load_type_during_the_week_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "day_of_the_week", y = "energy_usage_kWh", errorbar= None, hue = "load_type", palette = "YlGnBu")
plt.xlabel("Day of the Week")
plt.ylabel("Energy Usage (kWh)")
plt.title("Energy Consumption by Load Type During the Week", size = 12)

In [ ]:
consumption_by_load_type_by_weekstatus_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "week_status", y = "energy_usage_kWh", errorbar= None, hue = "load_type", palette = "YlGnBu")
plt.xlabel("Week Status")
plt.ylabel("Energy Usage (kWh)")
plt.title("Energy Consumption by Load Type and Week Status", size = 12)

Distribution of load type. counts
co2 emission during the week
co2 emission by load type

* **Load type distribution**

In [ ]:
load_type_distribution = energy_consumption["load_type"].value_counts()
load_type_distribution

In [ ]:
labels = ["Light load", "Medium load", "Maximum load"]
colors = ["#c7e9b4", "#41b6c4", "#225ea8"]

load_type_distribution_graph = plt.figure(figsize = (11, 3))
plt.pie(load_type_distribution, labels = labels, colors = colors, startangle = 265, autopct="%1.1f%%", shadow = True)
plt.axis("equal")
plt.legend()
plt.title("Load Type Distribution")

* **CO<sub>2</sub> emissions distribution during the week**

CO<sub>2</sub> emissions are higher on weekdays, particularly on Tuesdays and Thursdays. This trend is expected, as CO<sub>2</sub> emissions are closely linked to energy consumption.

In [ ]:
co2_emissions_during_the_week_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "day_of_the_week", y = "CO2_ppm", errorbar= None, hue = "day_of_the_week", palette = "YlGnBu")
plt.xlabel("Day of the Week")
plt.ylabel("CO$_2$ Emissions (ppm)")
plt.title("CO$_2$ Emissions During the Week", size = 12)

* **CO<sub>2</sub> emission by load type**

 CO<sub>2</sub> emissions are higher during peak load times and lower during lighter load periods.

In [ ]:
co2_emissions_by_load_type_graph = plt.figure(figsize = (11, 3))
sns.barplot(energy_consumption, x = "load_type", y = "CO2_ppm", errorbar= None, hue = "load_type", palette = "YlGnBu")
plt.xlabel("Load Type")
plt.ylabel("CO$_2$ Emissions (ppm)")
plt.title("CO$_2$ Emissions by Load Type", size = 12)

## Pre-model data transformation

* **Encoding categorical data**

Most machine learning models require numerical input, which means categorical variables must be converted into numerical values for better comprehension by the model. In the case of linear regression, dummy encoding is the preferred method for encoding categorical data, as it helps prevent multicollinearity issues in the model.

In [ ]:
energy_consumption = pd.get_dummies(energy_consumption, drop_first = True)
energy_consumption.head()

* **Split data**

The dataset is divided into two main parts: the training set (70%) and the testing set (30%). This is done to avoid overfitting by training on one set and testing in another, and to perform the evaluation of the model. 

**_Training set_**: this is the portion of the dataset used to train the model. During training, the model learns patterns, relationships, and features from the data.

**_Testing set_**: this set is used to evaluate the model's performance after training. It helps assess how well the model generalizes to unseen data.

In [ ]:
# Independent features
X = energy_consumption.drop("energy_usage_kWh", axis = 1)

# Dependent or target feature
y = energy_consumption["energy_usage_kWh"]

# Split the dataset into training and testing sets
# Import required function form the module
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print("Training set - X_train shape:", X_train.shape)
print("Testing set - X_test shape:", X_test.shape)
print("Training set - y_train shape:", y_train.shape)
print("Testing set - y_test shape:", y_test.shape)

## Linear Regression Model

This is a regression problem due to the continuous (numerical) nature of the target variable and the linear relationship between the features and the target variable. Therefore, a linear regression model is the most suitable choice. The LinearRegression module from scikit-learn will be used to build this model.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the linear regression model
linear_regression_model = LinearRegression()
# Train the model with the training set
linear_regression_model.fit(X_train, y_train)
# Make predictions using the model on test set
y_pred = linear_regression_model.predict(X_test)

The evaluation of the model will be conducted using the following metrics: coefficient of determination (R²), mean squared error (MSE), root mean squared error (RMSE), and mean absolute error (MAE).

In [ ]:
# Evaluate the model
r2_score = linear_regression_model.score(X_test, y_test) # coefficient of determination or r-squared
mse = mean_squared_error(y_test, y_pred) # mean squared error
rmse = np.sqrt(mse) #root mean squared error
mae = mean_absolute_error(y_test, y_pred) # mean absolute error

In [ ]:
# Print the results
print("Evaluation metrics of the regression model:\n")
print(f"R² Score: {r2_score:.3f}")
print(f"Mean Squared Error: {mse:.3f}")
print(f"Root Mean Squared Error: {rmse:.3f}")
print(f"Mean Absolute Error: {mae:.3f}")

After evaluation, the model is visualized by plotting the actual values against the predicted values.

In [ ]:
# Assigning actual and predicted values
x = y_test 
y = y_pred

# Calculate the parameters of the linear regression
m, b = np.polyfit(x, y, 1)

plt.figure(figsize = (11, 6))

# Scatter plot of actual vs. predicted values
plt.scatter(x, y, color = "#41b6c4", label = "Predicted vs Actual Values", alpha = 0.7, edgecolors = "white")

# Plot the linear regression line
plt.plot(x, m * x + b, color = "#0c2c84", label = f"Linear Fit: y = {m:.2f}x + {b:.2f}")

plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Linear Regression Model Visualization")
plt.grid(alpha = 0.3)
plt.legend()